In [1]:
from supabase import create_client
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from dotenv import load_dotenv
import requests
import os
from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders.recursive_url_loader import RecursiveUrlLoader
import re
from trafilatura import fetch_url, extract

load_dotenv()

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SECRET_KEY")

supabase = create_client(supabase_url, supabase_key)

embeddings_model = OpenAIEmbeddings(openai_api_key=os.environ.get("OPENAI_API_KEY"))


2023-09-18 13:53:22,308:INFO - NumExpr defaulting to 8 threads.


In [2]:
urls = []
docs = []

with open("sidebar.html", "r") as f:
    content = f.read()
    pattern = r'href="([^"]+)"'
    matches = re.findall(pattern, content)
    for match in matches:
        url = "https://mantine.dev" + match
        urls.append("https://mantine.dev" + match)
        downloaded = fetch_url(url)
        if not(downloaded):
            print("Failed load of: " + url)
            continue
        result = extract(downloaded, include_links=True, include_tables=True)
#         print(result)
        docs.append(Document(page_content=result, metadata={"title": url,"source": url}))
        

2023-09-18 13:53:28,165:ERROR - download error: https://mantine.devhttps://ui.mantine.dev HTTPSConnectionPool(host='mantine.devhttps', port=443): Max retries exceeded with url: //ui.mantine.dev (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x140f9ce10>: Failed to resolve 'mantine.devhttps' ([Errno 8] nodename nor servname provided, or not known)"))


Failed load of: https://mantine.devhttps://ui.mantine.dev


2023-09-18 13:53:28,436:WARNING - missing link attribute: 91% of browsers {}
2023-09-18 13:53:28,437:WARNING - missing link attribute: focus-visible polyfill {}
2023-09-18 13:53:28,437:WARNING - missing link attribute: fallback {}
2023-09-18 13:53:28,687:WARNING - missing link attribute: colors generator tool {}
2023-09-18 13:53:29,177:WARNING - missing link attribute: data-* attributes {}
2023-09-18 13:53:29,178:WARNING - missing link attribute: Portal {}
2023-09-18 13:53:29,940:WARNING - missing link attribute: theme {}
2023-09-18 13:53:30,043:WARNING - missing link attribute: better performance {}
2023-09-18 13:53:30,321:WARNING - missing link attribute: Input {}
2023-09-18 13:53:30,397:WARNING - missing link attribute: Tabs {}
2023-09-18 13:53:39,523:WARNING - missing link attribute: this guide {}
2023-09-18 13:53:45,167:WARNING - missing link attribute: this guide {}
2023-09-18 13:53:45,366:WARNING - missing link attribute: this guide {}
2023-09-18 13:53:45,945:WARNING - missing l

In [3]:
print(docs)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=17000)
docs = text_splitter.split_documents(docs)

docs_strings = []
for doc in docs:
    docs_strings.append(doc.page_content)
embeddings_vector = embeddings_model.embed_documents(docs_strings)

[Document(page_content='Getting started\n[Get started with a template](#get-started-with-a-template)\nThe easiest way to get started is to use one of the templates.\nAll templates include required dependencies and pre-configured settings.\nSome templates also include additional features like\n[Jest](https://jestjs.io/),\n[Storybook](https://storybook.js.org/), [ESLint](https://eslint.org/), etc.\nAll templates include only\n@mantine/core and\n@mantine/hooks packages,\nif you need additional\n@mantine/* packages follow installation instructions\nof the package you want to use.\nTo get started with a template, open it on GitHub and click on the "Use this template"\nbutton. Note that you need to be logged in to your GitHub account to use this feature.\nIf you are not familiar with GitHub, you can find a detailed instruction on how to\nbootstrap a project from a template\n[on this page](/templates-usage).\nTemplates list:\nnext-app-template\nNext.js template with app router and full setup:

In [4]:
data = []

for i in range(len(docs)):
    data.append({
        "embedding": embeddings_vector[i],
        "metadata": docs[i].metadata,
        "content": docs[i].page_content,
        "repo": "mantine"
    })

print(data[:10])


[{'embedding': [-0.0033856507137733018, 0.01778280187391915, -0.004616952064575733, -0.011754732652825726, -0.00950790761422756, 0.018632210908142138, 0.0048258798500031095, -0.008507796616646591, -0.008562596568603746, -0.01641278537986126, 0.017289595787046697, 0.009583258130245262, -0.01848150987610674, -0.0055348626685596805, -0.008535197058286459, 0.013426152142276996, 0.030441744444247597, -0.006733626169537753, 0.02672900276283588, -0.013864557345869714, 0.002017347853263468, -0.0012672643722470964, -0.009398306778990671, -0.021029738841420863, -0.014823568146652172, 0.005695839448286644, 0.020920137074861393, -0.017577298095958854, -0.029948538357375147, 0.00075650541586997, 0.030249942284091107, -0.009651759700006219, 0.006213020106686723, -0.028660724119559437, 0.01964602259220433, 0.0003733292129056943, 0.005928742270902937, -0.004637502628636279, 0.014398862698218098, -0.003921669466839096, 0.0003519227342898704, 0.004839579605161753, -0.005065632550198725, 0.00826804345078

In [8]:
for i in range(0, len(data), 10):
    supabase.table("documents").insert(data[i:i+10]).execute()

2023-09-18 14:06:17,195:INFO - HTTP Request: POST https://fmyzoqfdmuxtujffwngp.supabase.co/rest/v1/documents "HTTP/1.1 201 Created"
2023-09-18 14:06:17,680:INFO - HTTP Request: POST https://fmyzoqfdmuxtujffwngp.supabase.co/rest/v1/documents "HTTP/1.1 201 Created"
2023-09-18 14:06:18,110:INFO - HTTP Request: POST https://fmyzoqfdmuxtujffwngp.supabase.co/rest/v1/documents "HTTP/1.1 201 Created"
2023-09-18 14:06:18,625:INFO - HTTP Request: POST https://fmyzoqfdmuxtujffwngp.supabase.co/rest/v1/documents "HTTP/1.1 201 Created"
2023-09-18 14:06:19,055:INFO - HTTP Request: POST https://fmyzoqfdmuxtujffwngp.supabase.co/rest/v1/documents "HTTP/1.1 201 Created"
2023-09-18 14:06:19,317:INFO - HTTP Request: POST https://fmyzoqfdmuxtujffwngp.supabase.co/rest/v1/documents "HTTP/1.1 201 Created"
2023-09-18 14:06:19,739:INFO - HTTP Request: POST https://fmyzoqfdmuxtujffwngp.supabase.co/rest/v1/documents "HTTP/1.1 201 Created"
2023-09-18 14:06:20,143:INFO - HTTP Request: POST https://fmyzoqfdmuxtujffwn